## Atividade

Realizar 3 consultas levando em conta as tabelas Clientes, Vendas, Itens Venda, Produtos, Vendedores:

- 1. Criar um banco de dados no DW chamado VendasVarejo, e guartar as 5 tabelas no banco.

- 2. Consulta que mostre cada item vendido, com o Nome do Cliente, Data da Venda, Produto, Vendedor e Valor total do item.

In [2]:
# Carregando libs e iniciando Session
from pyspark.sql import SparkSession, functions as func
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

### Atividade 1

In [4]:
# Ver bancos de dados
spark.sql('show databases').show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [5]:
# Criar Novo bancos de dados
spark.sql('create database vendasvarejo')

DataFrame[]

In [6]:
# Usar o banco de dados recem-criado
spark.sql('use vendasvarejo')

DataFrame[]

In [7]:
# Importando e salvando tabela clientes
clientes = spark.read.format('parquet').load('../arquivos/Atividades/Clientes.parquet')
clientes.write.mode('overwrite').saveAsTable('clientes')

In [8]:
# Importando e salvando tabela vendas
vendas = spark.read.format('parquet').load('../arquivos/Atividades/Vendas.parquet')
vendas.write.mode('overwrite').saveAsTable('vendas')

In [12]:
# Importando e salvando tabela vendedores
vendedores = spark.read.format('parquet').load('../arquivos/Atividades/Vendedores.parquet')
vendedores.write.mode('overwrite').saveAsTable('vendedores')

In [13]:
# Importando e salvando tabela produtos
prod = spark.read.format('parquet').load('../arquivos/Atividades/Produtos.parquet')
prod.write.mode('overwrite').saveAsTable('produtos')

In [14]:
# Importando e salvando tabela itens vendas
itens_vendas = spark.read.format('parquet').load('../arquivos/Atividades/ItensVendas.parquet')
itens_vendas.write.mode('overwrite').saveAsTable('itensvendas')

In [16]:
# Checando as tabelas salvas
spark.sql('show tables').show()

+------------+-----------+-----------+
|   namespace|  tableName|isTemporary|
+------------+-----------+-----------+
|vendasvarejo|   clientes|      false|
|vendasvarejo|itensvendas|      false|
|vendasvarejo|   produtos|      false|
|vendasvarejo|     vendas|      false|
|vendasvarejo| vendedores|      false|
+------------+-----------+-----------+



### Atividade 2

In [22]:
# Checando tabela ItensVendas
spark.sql('select * from itensvendas limit 2').show()

+---------+--------+----------+-------------+----------+--------+----------------+
|ProdutoID|VendasID|Quantidade|ValorUnitario|ValorTotal|Desconto|TotalComDesconto|
+---------+--------+----------+-------------+----------+--------+----------------+
|        2|     400|         2|       9201.0|   18402.0| 6256,68|        12145.32|
|        2|     385|         2|       9201.0|   18402.0| 5704,62|        12697.38|
+---------+--------+----------+-------------+----------+--------+----------------+



In [23]:
# Checando tabela Clientes
spark.sql('select * from clientes limit 2').show()

+---------+--------------------+------+------+------+
|ClienteID|             Cliente|Estado|Genero|Status|
+---------+--------------------+------+------+------+
|        1|Adelina Buenaventura|    RJ|     M|Silver|
|        2|        Adelino Gago|    RJ|     M|Silver|
+---------+--------------------+------+------+------+



In [25]:
# Checando tabela Produtos
spark.sql('select * from produtos limit 2').show()

+---------+--------------------+---------+
|ProdutoID|             Produto|    Preco|
+---------+--------------------+---------+
|        1|Bicicleta Aro 29 ...|8.852,00 |
|        2|Bicicleta Altools...|9.201,00 |
+---------+--------------------+---------+



In [26]:
# Checando tabela Vendedores
spark.sql('select * from vendedores limit 2').show()

+----------+----------------+
|VendedorID|        Vendedor|
+----------+----------------+
|         1|    Armando Lago|
|         2|Capitolino Bahía|
+----------+----------------+



In [27]:
# Checando tabela Vendas
spark.sql('select * from vendas limit 2').show()

+--------+----------+---------+--------+------+
|VendasID|VendedorID|ClienteID|    Data| Total|
+--------+----------+---------+--------+------+
|       1|         1|       91|1/1/2019|8053.6|
|       2|         6|      185|1/1/2020| 150.4|
+--------+----------+---------+--------+------+



In [45]:
# Consulta
spark.sql('select c.Cliente as Nome,\
                  v.Data,\
                  p.Produto,\
                  d.Vendedor,\
                  i.ValorTotal\
                  from vendas v\
                  left join clientes c on v.ClienteID == c.ClienteID\
                  left join vendedores d on d.VendedorID == v.VendedorID\
                  left join itensvendas i on i.VendasID == v.VendasID\
                  left join produtos p on i.ProdutoID == p.ProdutoID\
                  sort by ValorTotal desc').show()

+--------------------+----------+--------------------+----------------+----------+
|                Nome|      Data|             Produto|        Vendedor|ValorTotal|
+--------------------+----------+--------------------+----------------+----------+
|    Antão Corte-Real|  4/1/2020|Bicicleta Altools...|  Hélio Liberato|   18402.0|
|    Antão Corte-Real| 10/4/2019|Bicicleta Altools...| Godo Capiperibe|   18402.0|
|      Cosme Zambujal| 17/5/2019|Bicicleta Altools...|Jéssica Castelão|   18402.0|
|      Brígida Gusmão|  4/6/2020|Bicicleta Altools...|  Hélio Liberato|   18402.0|
|   Bárbara Magalhães|15/12/2020|Bicicleta Altools...|  Hélio Liberato|   18402.0|
|    Humberto Almeida|28/12/2019|Bicicleta Altools...|   Iberê Lacerda|   18402.0|
|    Davide Alcántara| 24/2/2019|Bicicleta Aro 29 ...|Jéssica Castelão|   17704.0|
|    Antão Corte-Real| 23/3/2020|Bicicleta Aro 29 ...|  Tobias Furtado|   17704.0|
|       Cosme Ipanema|  5/4/2020|Bicicleta Altools...|  Tobias Furtado|   16561.8|
|  G